In [1]:
#This notebook trains a single RNN using all ten sessions of data and a specified train/test partition
#(can be 'HeldOutBlocks' or 'HeldOutTrials'). 
#The RNN training process is followed carefully in the notebook.

In [2]:
import numpy as np
import scipy.io
import scipy.ndimage.filters
import os
import matplotlib
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
from charSeqRNN import charSeqRNN, getDefaultRNNArgs

#point this towards the top level dataset directory
rootDir = os.path.join(os.path.expanduser('~'), 'handwritingBCIData')

#train an RNN using data from these specified sessions
dataDirs = ['t5.2019.05.08','t5.2019.11.25','t5.2019.12.09','t5.2019.12.11','t5.2019.12.18',
            't5.2019.12.20','t5.2020.01.06','t5.2020.01.08','t5.2020.01.13','t5.2020.01.15']

#use this train/test partition 
cvPart = 'HeldOutTrials'

#name of the directory where this RNN run will be saved
rnnOutputDir = cvPart

#all RNN runs are saved in 'Step4_RNNTraining'
data_path = os.path.join(rootDir, 'RNNTrainingSteps', 'Step4_RNNTraining')
if not os.path.isdir(data_path):
    os.mkdir(data_path)

In [3]:
#We will use the default arguments specified here
args = getDefaultRNNArgs()

#Configure the arguments for a multi-day RNN (that will have a unique input layer for each day)
for x in range(len(dataDirs)):
    args['sentencesFile_'+str(x)] = rootDir+'Datasets/'+dataDirs[x]+'/sentences.mat'
    args['singleLettersFile_'+str(x)] = rootDir+'Datasets/'+dataDirs[x]+'/singleLetters.mat'
    args['labelsFile_'+str(x)] = rootDir+'RNNTrainingSteps/Step2_HMMLabels/'+cvPart+'/'+dataDirs[x]+'_timeSeriesLabels.mat'
    args['syntheticDatasetDir_'+str(x)] = rootDir+'RNNTrainingSteps/Step3_SyntheticSentences/'+cvPart+'/'+dataDirs[x]+'_syntheticSentences/'
    args['cvPartitionFile_'+str(x)] = rootDir+'RNNTrainingSteps/trainTestPartitions_'+cvPart+'.mat'
    args['sessionName_'+str(x)] = dataDirs[x]
    
args['outputDir'] = os.path.join(data_path, rnnOutputDir)
if not os.path.isdir(args['outputDir']):
    os.mkdir(args['outputDir'])
    
#this weights each day equally (0.1 probability for each day) and allocates a unique input layer for each day (0-9)
args['dayProbability'] = '[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]'
args['dayToLayerMap'] = '[0,1,2,3,4,5,6,7,8,9]'

#save the arguments dictionary so that the RNN program can load it
pickle.dump( args, open(os.path.join(args['outputDir'], 'args.p'), "wb" ) )

In [7]:
argsFile = os.path.join(args['outputDir'], 'args.p')
argDict = pickle.load(open(argsFile, "rb" ))

#set the visible device to the gpu specified in 'args' (otherwise tensorflow will steal all the GPUs)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
print('Setting CUDA_VISIBLE_DEVICES to ' + argDict['gpuNumber'])
os.environ["CUDA_VISIBLE_DEVICES"]=argDict['gpuNumber']

Setting CUDA_VISIBLE_DEVICES to 0
